In [47]:
!pip install -qU langchain langchain-core langchain-community langchain-experimental langchain-openai langchain-huggingface langchain-groq langchain-cohere cohere pypdf langchain-text-splitters langchain-chroma rank_bm25 flashrank sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00


In [23]:
import os
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['COHERE_API_KEY'] = userdata.get('COHERE_API_KEY')

# Setting Up Langsmith

In [24]:
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGSMITH_TRACING'] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Normal_Rag"

# Set Up LLM

In [25]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model = "llama-3.2-1b-preview",
    temperature=0.1
)

# Load PDF

In [26]:
from langchain_community.document_loaders import PyPDFLoader

In [27]:
file_path = '/content/drive/MyDrive/Colab_Notebooks/GenerativeAI/RAG/Hybrid_RAG/Knowledge/HR.pdf'
loader = PyPDFLoader(
    file_path=file_path
)

In [28]:
docs = loader.load()

# Chunk Docs

In [29]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [30]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 300,
    length_function = len,
    is_separator_regex=False

)

In [31]:
texts = text_splitter.split_documents(docs)

# Embeddings

In [ ]:
# from langchain_openai import OpenAIEmbeddings

In [ ]:
# embeddings = OpenAIEmbeddings()

In [ ]:
# embeddings_size = embeddings.embed_query("Hello World")

# len(embeddings_size)

1536

In [46]:
# from langchain_community.embeddings import CohereEmbeddings

In [ ]:
# embeddings = CohereEmbeddings(model="embed-english-v3.0")

In [32]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [35]:
embeddings = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/114k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [36]:
embeddings_size = embeddings.embed_query("Hello World")

len(embeddings_size)

1024

# Vector Store

In [37]:
from langchain_chroma import Chroma

In [38]:
persist_directory='hr_db'

if os.path.exists(persist_directory):
    # Load from disk
    db = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings
    )
else:
  # Save to disk.
  db = Chroma.from_documents(
      documents=texts,
      embedding=embeddings,
      persist_directory=persist_directory
  )

In [39]:
vectorstores_retriever = db.as_retriever(search_kwargs={"k": 5})

# Keyword Store

In [40]:
from langchain_community.retrievers import BM25Retriever

In [41]:
keyword_retriever = BM25Retriever.from_documents(
    documents=docs,
)

In [42]:
keyword_retriever.k = 5

# Ensemble Retriever

In [43]:
from langchain.retrievers import EnsembleRetriever

In [44]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[keyword_retriever, vectorstores_retriever], weights=[0.3, 0.7]
)

# Reranking

In [48]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

In [49]:
compressor = CohereRerank(model="rerank-english-v3.0")

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever
)

In [50]:
compressed_docs = compression_retriever.get_relevant_documents('How many earned leaves do i get per quarter?')
compressed_docs

<ipython-input-50-dc14aca695ad>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  compressed_docs = compression_retriever.get_relevant_documents('How many earned leaves do i get per quarter?')


[Document(metadata={'page': 0, 'source': '/content/drive/MyDrive/Colab_Notebooks/GenerativeAI/RAG/Hybrid_RAG/Knowledge/HR.pdf', 'relevance_score': 0.9934865}, page_content='deep customer insight to develop innovative solutions.* Unity-We will invest in our people and partners, enable continuous learning, and build caring and collaborative relationships based on trust and mutual respect.Leave Policies:Company has a variety of leave policies, including paid leave, leave without pay, and additional leave:  Paid leaveCompany provides a minimum of 12 paid days per year, which can be used for vacation or sick leave.  Earned leavesEmployees receive 16 earned leaves per year, or four per quarter. These leaves can accumulate up to 48, after which they can be cashed in.  Casual leavesEmployees receive seven casual leaves per year, or 1.75 per quarter. Casual leaves expire at the end of the financial year.  Sick leavesEmployees receive 10 sick leaves per year, or 2.5 per quarter. Sick leaves can 

# Chain With Memory

In [51]:
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

## History Aware Retriever

In [52]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question which might reference context in the chat history, "
    "formulate a standalone question which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
)

In [53]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [54]:
history_aware_retriever = create_history_aware_retriever(
    llm,
    compression_retriever,
    contextualize_q_prompt
)

## Answer Question

In [55]:
system_prompt = (
    "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question."
    "If you don't know the answer, say that you don't know."
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

In [56]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# RAG Chain

In [57]:
qa_chain = create_stuff_documents_chain(llm, qa_prompt)

In [58]:
rag_chain = create_retrieval_chain(
    history_aware_retriever,
    qa_chain
)

# Chat History

In [59]:
from langchain_core.messages import HumanMessage, AIMessage

In [60]:
chat_history = []

In [61]:
user_input = "How many earned leaves do i get per quarter?"

response = rag_chain.invoke({
    "input" : user_input,
    "chat_history" : chat_history
})


chat_history.extend([
    HumanMessage(content=user_input),
    AIMessage(content=response['answer'])
])

print(response['answer'])

According to the provided context, employees receive 16 earned leaves per year, or four per quarter.


In [62]:
user_input = "Can I wear jersey on monday?"

response = rag_chain.invoke({
    "input" : user_input,
    "chat_history" : chat_history
})


chat_history.extend([
    HumanMessage(content=user_input),
    AIMessage(content=response['answer'])
])

print(response['answer'])

According to the provided context, on Fridays, employees can wear casual, half-sleeved shirts, collared t-shirts, golf/polo shirts, turtlenecks, trousers, jeans, and casual shoes.


In [63]:
chat_history

[HumanMessage(content='How many earned leaves do i get per quarter?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='According to the provided context, employees receive 16 earned leaves per year, or four per quarter.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Can I wear jersey on monday?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='According to the provided context, on Fridays, employees can wear casual, half-sleeved shirts, collared t-shirts, golf/polo shirts, turtlenecks, trousers, jeans, and casual shoes.', additional_kwargs={}, response_metadata={})]

In [64]:
user_input = "Does that come under casuals?"

response = rag_chain.invoke({
    "input" : user_input,
    "chat_history" : chat_history
})


chat_history.extend([
    HumanMessage(content=user_input),
    AIMessage(content=response['answer'])
])

print(response['answer'])

Yes, the provided context does mention "smart casuals" on Fridays, which includes casual, half-sleeved shirts, collared t-shirts, golf/polo shirts, turtlenecks, trousers, jeans, and casual shoes.
